In [13]:
%load_ext autoreload

import graphistry
import neo4j
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
%autoreload

def test_hey():
    driver = neo4j.GraphDatabase.driver("bolt://neo4j:7687")
    # MATCH (a)-[r:TRANSACTION]->(b) WHERE r.USD > 7000 AND r.USD < 10000  RETURN a, r, b ORDER BY r.USD DESC LIMIT 5
    with driver.session() as session:
        statement = session.run("MATCH (a)-[r:TRANSACTION]->(b) RETURN a, r, b LIMIT 1", { })

        graph = statement.graph()

        for relationship in graph.relationships: # type: neo4j.Relationship
            print(json.dumps({
                key: value for key, value in _relationship_fields(relationship)
            }, indent=4))

        for node in graph.nodes: # type: neo4j.Node
            print(json.dumps({
                key: value for key, value in _node_fields(node)
            }, indent=4))

def _node_fields(node: neo4j.Node):
    yield ("__NODE_ID__", node.id)
    yield ("__NEO4J_LABEL__", list(node.labels))
    for index, label in enumerate(node.labels):
        yield (f"__NEO4J_LABEL_{index}__", label)
    for item in node.items():
        yield item

def _relationship_fields(relationship: neo4j.Relationship):
    yield ("__EDGE_ID__", relationship.id)
    yield ("__EDGE_SRC__", relationship.start_node.id)
    yield ("__EDGE_DST__", relationship.end_node.id)
    yield ("__NEO4J_TYPE__", relationship.type)
    for item in relationship.items():
        yield item

test_hey()

{
    "__EDGE_ID__": 39369,
    "__EDGE_SRC__": 0,
    "__EDGE_DST__": 275,
    "__NEO4J_TYPE__": "TRANSACTION",
    "_title": "bb4f7cec835dbc9670c45746bd4c05a3",
    "edgeColor": "-5845277",
    "edgeColor2": "0",
    "Amount $": "4960",
    "Date": "2013-11-27T03:18:48.000Z",
    "Transaction ID": "bb4f7cec835dbc9670c45746bd4c05a3ac80cede11972283aabae296"
}
{
    "__NODE_ID__": 0,
    "__NEO4J_LABEL__": [
        "ACCOUNT"
    ],
    "__NEO4J_LABEL_0__": "ACCOUNT",
    "Account": "2dd13954e18508bb8b3a41d96a022be9",
    "degree_in": "1",
    "Balance (max) $": "8680",
    "degree": "17",
    "_title": "2dd13954e18508bb8b3a41d96a022be9",
    "Balance (avg) $": "-2134.91609968",
    "__defaultPointSize": "5",
    "Tainted Coins": "0",
    "__pointSizes": "5",
    "x": "2343.969482421875",
    "y": "1269.2396240234375",
    "Balance $": "-106745.80498400002",
    "__pointColors": "14929574",
    "degree_out": "16",
    "__pointCommunity": "275",
    "pointColor": "-5845277"
}
{
    "__NO